# MCMC Inference with AGETs

This code shows how to use the AGETs inferred in the Notebook *Creating Approximated Gene Expression Trajectories (AGETs)* in an MCMC approach to infer GRN parameters. The AGETs contain a target gene expression for the Tbox genes that can be fit to. It also contains initial conditions (Tbox expression at the start) and boundary conditions (Wnt and FGF expression over time). For the inference, we only use a subset of all 1903 AGETs because of computation time. Only AGETs that are observed from start to end of the movie are used (longest AGETs), to minimize the influence of the initial conditions on the whole simulated expression. A randomly chosen subset of these longest AGETs are saved in the file *List_of_100_cell_tracks.txt*. For the following inference, 10 of these AGETs where chosen semi-randomly, meaning that they where chosen to approximately represent the whole region of interest in the PSM. Practically that meant, that the first 10 AGETs did not fulfill that criterion by visual inspection and the second 10 AGETs did. Therefore, AGETs 10-19 were used. In the code, Tbxta is referred to as g1, Tbx16 is referred to as g2 and Tbx24 is referred to as g3.

## Define simulator

In the following, a simulator class *Simulate_on_tracks* is defined. As indicated by the file name, tracks and AGETs are used interchangeably in the code. As explained in the paper, the simulator takes as input a set of 24 parameters describing the GRN and use ODEs with the initial and boundary conditions from the AGETs to simulate gene expression for the chosen 10 AGETs. The simulator class is then used in the following MCMC inference. 

In [1]:
import numpy as np
import pandas as pd
import pickle
from scipy.integrate import odeint
import emcee
from multiprocessing import Pool
from multiprocessing import cpu_count 
import time

In [2]:
# Define simulator class 

class Simulate_on_tracks:
    '''
    The simulator takes as input a set of 24 parameters describing the GRN 
    and uses ODEs with the initial and boundary conditions from the AGETs to 
    simulate gene expression for the chosen 10 AGETs. The simulated gene expression
    is then added to the AGET, so that the AGET includes the original target as well 
    as the simulated gene expression.
    '''

    def __init__(self, parameters):
        self.params = parameters
        with open("Dependencies_simulator/List_of_100_cell_tracks.txt", "rb") as fp:   
            self.List_of_100_cell_tracks = pickle.load(fp)[10:20] # Load the chosen 10 AGETs

    def simulation(self):
        params = self.params

        # Define helper functions for ODEs, see paper explanation of the ODEs
        def g(x): 
            return 0.5 * ((x / np.sqrt(x ** 2 + 1)) + 1)
        def PSH(s, t, B1, B2):
            B = [B1, B2]
            W = np.array(
                [
                    [params[0], params[1], params[2]],
                    [params[3], params[4], params[5]],
                    [params[6], params[7], params[8]],
                ]
            )
            E = np.array(
                [[params[18], params[17]], [params[19], params[15]], [params[20], params[16]]]
            )
            R = [params[9], params[10], params[11]]
            lmd = [params[12], params[13], params[14]]
            h = [params[21], params[22], params[23]]
            u = np.array(
                [
                    W[0].dot(s) + E[0].dot(B) + h[0],
                    W[1].dot(s) + E[1].dot(B) + h[1],
                    W[2].dot(s) + E[2].dot(B) + h[2],
                ]
            )
            d_tbxta_dt = R[0] * g(u[0]) - lmd[0] * s[0]
            d_tbx16_dt = R[1] * g(u[1]) - lmd[1] * s[1]
            d_tbx24_dt = R[2] * g(u[2]) - lmd[2] * s[2]
            dsdt = [d_tbxta_dt, d_tbx16_dt, d_tbx24_dt]
            return dsdt
        df_out = []
        
        # Iterate through celltracks (AGETs)
        for i_celltrack in range(len(self.list_of_cell_tracks)):
            df_celltrack = self.list_of_cell_tracks[i_celltrack]
            df_celltrack['Time_nPSM'] = np.nan
            df_celltrack.Time_nPSM = df_celltrack.Time*90/3600/3 # Add biological time, used later
            df_celltrack['g1_sim'] = df_celltrack['g2_sim'] = df_celltrack['g3_sim'] = np.nan # add empty columns for simulated gene expression 
            df_celltrack.loc[0, ['g1_sim', 'g2_sim', 'g3_sim']] = df_celltrack.loc[0, ['g1', 'g2', 'g3']].values # initial conditions are the same
            max_time = np.max(df_celltrack.Time)
            for index, row in df_celltrack.loc[0:df_celltrack.shape[0], :].iterrows(): # stepwise integration until last time point is reached
                if row['Time'] != max_time: 
                    B1 = row['Wnt']
                    B2 = row['FGF']
                    count_timesteps_between = df_celltrack.Time[index+1] - df_celltrack.Time[index] # number of time steps between observations, sometimes an observation at a specific time is missing
                    t_interval = np.linspace(df_celltrack.Time_nPSM[index], df_celltrack.Time_nPSM[index+1], 10*int(count_timesteps_between))
                    s0 = row[['g1_sim', 'g2_sim', 'g3_sim']]
                    s = odeint(PSH, s0, t_interval, args=(B1, B2))
                    df_celltrack.loc[index+1, ['g1_sim', 'g2_sim', 'g3_sim']] = s[-1]
            df_out.append(df_celltrack) # append AGET with the simulated expression to the output list
        return df_out


## Define MCMC building blocks and run MCMC

For the MCMC, a logprior function consisting of a Gaussian loglikelihood function and a logprior function is needed. For further guidance, a look at the tutorials for the used MCMC Python implementation [emcee](https://emcee.readthedocs.io/en/stable/) is recommended. The uniform prior was chosen based on biological intuition for reasonable values for the parameters. The MCMC inference should be run on a powerful computer. The computation time can be reduced by using more cores as the computation can be parallelized.

In [ ]:
# MCMC

def loglikelihood(theta):
    sim = Simulate_on_tracks(parameters=theta) # Simulate gene expression for parameters theta on above defined 10 AGETs
    data = sim.simulation() 
    g_sim_g1 = data[0].g1_sim.values # simulated expression for Tbxta for AGET 1
    g_target_g1 = data[0].g1.values # target expression for Tbxta from AGET 1
    g_sim_g2 = data[0].g2_sim.values
    g_target_g2 = data[0].g2.values
    g_sim_g3 = data[0].g3_sim.values
    g_target_g3 = data[0].g3.values
    for i_ncelltracks in range(1, len(data)): # combine the values for each gene for the 10 AGETs, used for ll calulation below
        g_sim_g1 = np.append(g_sim_g1, data[i_ncelltracks].g1_sim.values) 
        g_target_g1 = np.append(g_target_g1, data[i_ncelltracks].g1.values)
        g_sim_g2 = np.append(g_sim_g2, data[i_ncelltracks].g2_sim.values)
        g_target_g2 = np.append(g_target_g2, data[i_ncelltracks].g2.values)
        g_sim_g3 = np.append(g_sim_g3, data[i_ncelltracks].g3_sim.values)
        g_target_g3 = np.append(g_target_g3, data[i_ncelltracks].g3.values)
    # The likelihood is calculated based on the difference between simulated and target expression
    # The closer the simulation is to the target expression, the higher the likelihood
    ll1 = -0.5 * np.sum(((g_sim_g1 - g_target_g1)/0.2)**2) # SD=0.2 found by testing different SDs and looking what works best
    ll2 = -0.5 * np.sum(((g_sim_g2 - g_target_g2)/0.2)**2) # SD=0.2 found by testing different SDs and looking what works best
    ll3 = -0.5 * np.sum(((g_sim_g3 - g_target_g3)/0.1)**2) # SD=0.1 found by testing different SDs and looking what works best
    return ll1 + ll2 + ll3

def logprior(theta): # prior was chosen based on biological intuition for the parameters
    # This is a uniform prior, meaning that all values inside the prior have the same probability and all values outside
    # of the prior have probability 0 
    W11, W12, W13, W21, W22, W23, W31, W32, W33, r1, r2, r3, lmd1, lmd2, lmd3, f1, f2, f3, w1, w2, w3, H1, H2, H3 = theta
    if 0 < W11 < 20 and \
       -20  < W12 < 20  and \
       -20  < W13 < 20  and \
       -20  < W21 < 20  and \
       0  < W22 < 20  and \
       -20  < W23 < 20  and \
       -20 < W31 < 20 and \
       -20  < W32 < 20  and \
       0  < W33 < 30  and \
       0 < r1 < 20 and \
       0 < r2 < 20 and \
       0 < r3 < 20 and \
       0 < lmd1 < 10 and \
       0 < lmd2 < 10 and \
       0 < lmd3 < 10 and \
       -30 < f1 < 30 and \
       -30 < f2 < 30 and\
       -30 < f3 < 30 and\
       0 < w1 < 30 and \
       -30 < w2 < 30 and \
       -30 < w3 < 30 and \
       -30 < H1 < 30 and \
       -30 < H2 < 30 and \
       -30 < H3 < 30:
        lp = 0.
    else:
        lp = -np.inf
    return lp

def logposterior(theta):
    lp = logprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + loglikelihood(theta)

# Number of walkers, the MCMC method used uses multiple walkers. See the emcee documentation for more information.
# We used 70 walkers, but the documentation recommended to use several hundreds. In our case, this didn't lead 
# to better results and was therefore set to 70 to reduce computation time. With 20 cores, 70 walkers, 50000 steps and
# 10 AGETs, it took a approximately 3 days to run the MCMC.
nwalkers = 70 

# Initial (nwalkers) points, starting points for the walkers
W11_ini = np.random.uniform(0,20, nwalkers) 
W12_ini = np.random.uniform(-20,20, nwalkers) 
W13_ini = np.random.uniform(-20,20, nwalkers)
W21_ini = np.random.uniform(-20,20, nwalkers)
W22_ini = np.random.uniform(0,20, nwalkers) 
W23_ini = np.random.uniform(-20,20, nwalkers)
W31_ini = np.random.uniform(-20,20, nwalkers)
W32_ini = np.random.uniform(-20,20, nwalkers)
W33_ini = np.random.uniform(0,30, nwalkers)
r1_ini = np.random.uniform(0, 20, nwalkers)
r2_ini = np.random.uniform(0, 20, nwalkers)
r3_ini = np.random.uniform(0, 20, nwalkers)
lmd1_ini = np.random.uniform(0, 10, nwalkers) 
lmd2_ini = np.random.uniform(0, 10, nwalkers)
lmd3_ini = np.random.uniform(0, 10, nwalkers)
f1_ini = np.random.uniform(-30,30, nwalkers)
f2_ini = np.random.uniform(-30,30, nwalkers)
f3_ini = np.random.uniform(-30,30, nwalkers)
w1_ini = np.random.uniform(0,30, nwalkers)
w2_ini = np.random.uniform(-30,30, nwalkers)
w3_ini = np.random.uniform(-30,30, nwalkers)
H1_ini = np.random.uniform(-30,30, nwalkers)
H2_ini = np.random.uniform(-30,30, nwalkers)
H3_ini = np.random.uniform(-30,30, nwalkers)

# Combine initial points
p0 = np.array([W11_ini, W12_ini, W13_ini, \
                       W21_ini, W22_ini, W23_ini,\
                       W31_ini, W32_ini, W33_ini,\
                       r1_ini, r2_ini, r3_ini,\
                       lmd1_ini, lmd2_ini, lmd3_ini,\
                       f1_ini, f2_ini,f3_ini, w1_ini, w2_ini,\
                       w3_ini, H1_ini, H2_ini, H3_ini ]).T

# Define number of steps for the MCMC inference
Nsamples = 50000    # from Kay 
#Nsamples = 100 # to make it run faster 
ndim = p0.shape[1] # needed below
i_run = 1 # if running multiple times, i_run is used for the name of the saved results
ncpu = cpu_count() # finde number of available CPUs for parallel processing
print("{0} CPUs".format(ncpu))

# Run MCMC (this takes time)
with Pool(ncpu) as pool: 
    sampler = emcee.EnsembleSampler(nwalkers, ndim, logposterior, pool=pool)
    start = time.time()
    sampler.run_mcmc(p0, Nsamples, progress=True)
    end = time.time()
    multi_time = end - start
    print("Multiprocessing took {0:.1f} seconds".format(multi_time))

    
# Save samples    
samples = sampler.get_chain(flat=True)
print('samples')
print(samples.shape)
print(type(samples))
print(' ')
with open(f"samples_run{i_run}.txt", "wb") as fp:   #Pickling
    pickle.dump(samples, fp)

# Save log probabilities
log_probs = sampler.flatlnprobability
print('log_probs')
print(log_probs.shape)
print(type(log_probs))
print(' ')
with open(f"log_probs_run{i_run}.txt", "wb") as fp:   #Pickling
    pickle.dump(log_probs, fp)

# Save MAP params (params with highest posterior probability)
map_params = samples[np.argmax(sampler.flatlnprobability)]
print('map_params')
print(map_params.shape)
print(type(map_params))
print(' ')
with open(f"map_params_run{i_run}.txt", "wb") as fp:   #Pickling
    pickle.dump(map_params, fp)

# Save acceptance fraction: MCMC diagnostic
acceptance_fraction = sampler.acceptance_fraction
print('acceptance_fraction')
print(acceptance_fraction.shape)
print(type(acceptance_fraction))
print(' ')
with open(f"acceptance_fraction_run{i_run}.txt", "wb") as fp:   #Pickling
    pickle.dump(acceptance_fraction, fp)

# Save autocorrelation time: MCMC diagnostic
autocorr_time = sampler.get_autocorr_time(quiet=True)
print('autocorr_time')
print(autocorr_time.shape)
print(type(autocorr_time))
print(' ')
with open(f"autocorr_time_run{i_run}.txt", "wb") as fp:   #Pickling
    pickle.dump(autocorr_time, fp)


8 CPUs
emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:  params:emcee: Exception while calling your likelihood function:

 

[ 19.76710687  -7.34972624  11.63437276 -16.56560983   0.45229857
   6.78455867  -9.99942198 -10.91811799  22.72309513  14.39435779
  19.31212881   3.06104553   5.31331035   1.4296027    4.26703455
  20.46959047  17.3762571  -13.49434669  28.06946711  13.16970775
 -10.40761345  -8.10385582 -24.1793156   -3.34155485]  params:  params:  params:
 emcee: Exception while calling your likelihood function:  params:emcee: Exception while calling your likelihood function: [  4.07258038  12.54573472   8.69038704  -4.31952913   3.34488443
  10.50234259  -1.40282517 -11.24693491  17.3655927   11.51357705
  12.17628349   2.82182141   9.11963227   7.80938686   6.95305081
   3.83967077  18.20361219 -24.9

Traceback (most recent call last):



{}
  kwargs:

  File "/home/shannon/miniconda3/envs/AGET/lib/python3.7/site-packages/emcee/ensemble.py", line 619, in __call__
    return self.f(x, *self.args, **self.kwargs)


  kwargs:[]

...

AttributeError: 'Simulate_on_tracks' object has no attribute 'list_of_cell_tracks'


  exception:

Traceback (most recent call last):
AttributeError: 'Simulate_on_tracks' object has no attribute 'list_of_cell_tracks'
  File "/home/shannon/miniconda3/envs/AGET/lib/python3.7/site-packages/emcee/ensemble.py", line 619, in __call__
    return self.f(x, *self.args, **self.kwargs)


Traceback (most recent call last):
  File "<ipython-input-2-7da20e8b3404>", line 53, in simulation
    for i_celltrack in range(len(self.list_of_cell_tracks)):
Traceback (most recent call last):
AttributeError: 'Simulate_on_tracks' object has no attribute 'list_of_cell_tracks'
  File "<ipython-input-4-1954199b2d24>", line 63, in logposterior
    return lp + loglikelihood(theta)
  File "/home/shannon/miniconda3/envs/AGET/lib/python3.7/site-packages/emcee/ensemble.py", line 619, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/shannon/miniconda3/envs/AGET/lib/python3.7/site-packages/emcee/ensemble.py", line 619, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-4-1954199b2d24>", line 5, in loglikelihood
    data = sim.simulation()
  File "<ipython-input-4-1954199b2d24>", line 63, in logposterior
    return lp + loglikelihood(theta)
  File "<ipython-input-4-1954199b2d24>", line 63, in logposterior
    return lp + loglikelihood(thet

emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:  params:
emcee: Exception while calling your likelihood function:  params: 
 [  6.64791514  12.79896474 -10.35667997  15.11065786  12.6114737
  -3.00387079 -19.43840348  -2.12135156  23.93240365  18.06592327
   1.78879804  11.95506392   4.75617548   9.37497294   4.66256029
 -16.6567897   23.28414205  12.44771879   3.6915345  -28.8742497
  -3.4643432  -28.97550629 -22.72203659  19.77470538]  params:[  2.19930218 -11.65137114  11.64012904  -5.13223508   6.61840714
 -10.47462025  -0.55555626  17.05010731   1.72517508  10.80895118
  12.22647953   3.08595061   3.05093471   7.00127165   1.60449069
 -14.09441488 -24.61206227  24.59127536  18.75654774  -2.50069273
 -15.8383366   15.2383498  -21.11176254 -11.08204665]
emcee: Exception while calling your likelihood function:
   args:
  args:emcee: Exception while calling your likelihood function:[ 19.77664471  -8.43009735  12.0756338